# Documentation - Climate Change in France Visualisation

The goal of this notebook is to showcase the data sources as well as the process of data transformation related to the databases in order to construct the data visualisation dashboard in this project.


In [1]:
# Libraries
import pandas as pd
import requests
from zipfile import ZipFile
from pathlib import Path


## 1. Data Acquisition


### 1.1. Données changement climatique - Longues Séries Homogénéisées

[Source Page](https://www.data.gouv.fr/en/datasets/donnees-changement-climatique-lsh-longues-series-homogeneisees/#/resources)

Variables:

- TN: Température minimale/Minimal temperature
- TX: Température maximale/Maximal temperature
- RR: Précipitations/Precipitation


In [2]:
LSH_SOURCES = {
    "RR": {
        "dataset": "https://www.data.gouv.fr/fr/datasets/r/76fd5cf2-d936-47ca-b7a5-a4b6cc081963",
        "list": "https://www.data.gouv.fr/fr/datasets/r/4aa508b1-d7a8-48c4-a8de-6e76b8641cf6",
    },
    "TN": {
        "dataset": "https://www.data.gouv.fr/fr/datasets/r/f42d8312-1b56-4685-81a7-43f3fd3e12e9",
        "list": "https://www.data.gouv.fr/fr/datasets/r/e2fb0be7-1d32-4f44-944d-e97e5961ce5d",
    },
    "TX": {
        "dataset": "https://www.data.gouv.fr/fr/datasets/r/6c47e287-d1d0-4430-9f1a-c0d4f06e77c3",
        "list": "https://www.data.gouv.fr/fr/datasets/r/f6382e83-c207-4e33-b413-4d6acbb08bf4",
    },
}


In [3]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
if not RAW_DIR.exists():
    RAW_DIR.mkdir(parents=True)


In [4]:
LSH_DIR = RAW_DIR / "LSH"
LSH_ZIP_DIR = LSH_DIR / "zip"
if not LSH_ZIP_DIR.exists():
    LSH_ZIP_DIR.mkdir(parents=True)

for k, v in LSH_SOURCES.items():
    # Create directory
    dir_name = k

    dataset_dir = LSH_DIR / dir_name
    dataset_list_dir = LSH_DIR / "list"
    if not dataset_list_dir.exists():
        dataset_list_dir.mkdir()
    if dataset_dir.exists():
        continue
    dataset_dir.mkdir()

    # Download datasets
    url = v["dataset"]
    r = requests.get(url)
    with open(LSH_ZIP_DIR / f"{k}.zip", "wb") as f:
        f.write(r.content)
    # Unzip
    with ZipFile(LSH_ZIP_DIR / f"{k}.zip", "r") as zip_ref:
        zip_ref.extractall(dataset_dir)

    # Download list
    url = v["list"]
    r = requests.get(url)
    with open(dataset_list_dir / f"list_{k}.csv", "wb") as f:
        f.write(r.content)


### 1.2 Metadata on Meterological Stations

[Source Page](https://www.data.gouv.fr/en/datasets/informations-sur-les-stations-metadonnees/)


In [5]:
METADATA_DIR = RAW_DIR / "metadata"
if not METADATA_DIR.exists():
    METADATA_DIR.mkdir(parents=True)

METADATA_SOURCE = (
    "https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922"
)

r = requests.get(METADATA_SOURCE)
with open(METADATA_DIR / "metadata.geojson", "wb") as f:
    f.write(r.content)


### 1.3 Department Shapes

[Source Page](https://france-geojson.gregoiredavid.fr/)


In [6]:
DEP_DIR = RAW_DIR / "departments_shape"
if not DEP_DIR.exists():
    DEP_DIR.mkdir(parents=True)

DEP_SOURCE = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"

r = requests.get(DEP_SOURCE)
with open(DEP_DIR / "departments.geojson", "wb") as f:
    f.write(r.content)


## 2. Data Transformation


In [7]:
PROCESSED_DIR = DATA_DIR / "processed"
if not PROCESSED_DIR.exists():
    PROCESSED_DIR.mkdir(parents=True)


### 2.1 Extract all stations and their metadata


In [8]:
import json

with open("../data/raw/metadata/metadata.geojson", "r") as f:
    station_metadata = json.load(f)

station_metadata = pd.json_normalize(
    [v for _, v in station_metadata["features"].items()]
)
station_metadata_variables = {
    "properties.NUM_POSTE": "station_id",
    "properties.NOM_USUEL": "station_name",
    "properties.NUM_DEP": "department_id",
    "properties.ALTI": "altitude",
    "properties.LAT_DG": "latitude",
    "properties.LON_DG": "longitude",
    "properties.COMMUNE": "city",
}
station_metadata = station_metadata[station_metadata_variables.keys()]
station_metadata = station_metadata.rename(columns=station_metadata_variables)
station_metadata["station_id"] = station_metadata["station_id"].astype(int)
station_metadata.to_csv(PROCESSED_DIR / "station_metadata.csv", index=False)
station_metadata.head()


,station_id,station_name,department_id,altitude,latitude,longitude,city
0,1014002,ARBENT,1,534,46.278167,5.669000,ARBENT
1,1027003,BALAN_AERO,1,196,45.833000,5.106667,BALAN
2,1028001,BANEINS,1,243,46.122000,4.904500,BANEINS
3,1033002,BELLEGARDE,1,350,46.086500,5.814167,VALSERHONE
4,1034004,BELLEY,1,330,45.769333,5.688000,BELLEY


### 2.2 Time Series Aggregation


In [9]:
list_RR = pd.read_csv("../data/raw/LSH/list/list_RR.csv", sep=";", header=2)
list_TN = pd.read_csv("../data/raw/LSH/list/list_TN.csv", sep=";", header=2)
list_TX = pd.read_csv("../data/raw/LSH/list/list_TX.csv", sep=";", header=2)


def filter_df_list(df: pd.DataFrame, name: str) -> pd.DataFrame:
    df = df.copy()
    col_to_keep = ["nom_fichier", "num_poste"]
    df = df[col_to_keep]
    df.rename(
        columns={"nom_fichier": "file_name", "num_poste": "station_id"}, inplace=True
    )
    df["variable"] = name
    return df


list_RR = filter_df_list(list_RR, "RR")
list_TN = filter_df_list(list_TN, "TN")
list_TX = filter_df_list(list_TX, "TX")


In [10]:
df_list = pd.concat([list_RR, list_TN, list_TX])
df_list = df_list[["station_id", "variable", "file_name"]]
df_list = df_list.reset_index()
df_list.drop("index", axis=1, inplace=True)

df_list.to_csv(PROCESSED_DIR / "aggregated_list.csv", index=True)
df_list


,station_id,variable,file_name
0,1034004,RR,SH_MRR101034004.csv
1,1072001,RR,SH_MRR101072001.csv
2,1089001,RR,SH_MRR101089001.csv
3,1143002,RR,SH_MRR101143002.csv
4,1235001,RR,SH_MRR001235001.csv
...,...,...,...
1466,91103001,TX,SH_MTX291103001.csv
1467,91184001,TX,SH_MTX091184001.csv
1468,94068001,TX,SH_MTX394068001.csv
1469,95078001,TX,SH_MTX295078001.csv


In [11]:
def process_list_row(row: pd.Series) -> pd.DataFrame:
    station_id = row["station_id"]
    variable = row["variable"]
    file_name = row["file_name"]

    if variable == "IN":
        header = 11
    else:
        header = 12

    df = pd.read_csv(LSH_DIR / variable / file_name, sep=";", header=header)
    df = df[["YYYYMM", "VALEUR"]]
    df.columns = ["timestamp", "value"]
    df["station_id"] = station_id
    df["variable"] = variable

    return df


In [12]:
measurements = pd.DataFrame(columns=["timestamp", "value", "station_id", "variable"])
for row in df_list.iterrows():
    row_df = process_list_row(row[1])
    measurements = pd.concat([measurements, row_df], ignore_index=True)
measurements.head()


/tmp/ipykernel_83620/1611672627.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  measurements = pd.concat([measurements, row_df], ignore_index=True)


,timestamp,value,station_id,variable
0,195101,83.0,1034004,RR
1,195102,134.4,1034004,RR
2,195103,135.3,1034004,RR
3,195104,76.6,1034004,RR
4,195105,129.9,1034004,RR


In [13]:
measurements = measurements[["timestamp", "station_id", "variable", "value"]]
measurements = measurements.sort_values(["timestamp", "station_id", "variable"])
measurements["timestamp"] = pd.to_datetime(measurements["timestamp"], format="%Y%m")
measurements["station_id"] = measurements["station_id"].astype(str)
measurements


,timestamp,station_id,variable,value
315778,1945-01-01,35238003,RR,65.6
316714,1945-01-01,35281001,RR,69.8
317650,1945-01-01,35294001,RR,64.0
395699,1945-01-01,44015001,RR,64.5
397499,1945-01-01,44067001,RR,64.3
...,...,...,...,...
1004584,2022-12-01,95078001,TN,2.2
1234011,2022-12-01,95078001,TX,7.3
785711,2022-12-01,95088001,RR,55.4
1005424,2022-12-01,95088001,TN,2.9


### 2.3 Store data to a SQLite instance


In [14]:
import sqlite3

if (DATA_DIR / "data.db").exists():
    (DATA_DIR / "data.db").unlink()

conn = sqlite3.connect(DATA_DIR / "data.db")
measurements.to_sql(
    "measurements", conn, if_exists="fail", index=True, index_label="id"
)
station_metadata.to_sql(
    "stations", conn, if_exists="fail", index=True, index_label="id"
)


2394